In [1]:
# Dependencies
import json
import requests
from api_keys import api_key
import pandas as pd
from uszipcode import SearchEngine
from census import Census
from api_keys import census_key
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [2]:
c = Census(census_key, year=2015)
census_data = c.acs5.get("B19013_001E",
                          {'for': 'zip code tabulation area:*'})
census_df = pd.DataFrame(census_data)
census_df=census_df['zip code tabulation area']
census_df=pd.DataFrame(census_df)
census_df.rename(columns={'zip code tabulation area':'Zipcode'}, inplace=True)

In [3]:
census_df

,Zipcode
0,12810
1,12811
2,12812
3,12814
4,12815
...,...
33115,71404
33116,28735
33117,34139
33118,40007


In [4]:
#Searching by zipcode
search = SearchEngine()
results = search.by_zipcode("92122")
results

SimpleZipcode(zipcode='92122', zipcode_type='Standard', major_city='San Diego', post_office_city='San Diego, CA', common_city_list=['San Diego'], county='San Diego County', state='CA', lat=32.86, lng=-117.2, timezone='Pacific', radius_in_miles=2.0, area_code_list=['619', '760', '858'], population=43728, population_density=2745.0, land_area_in_sqmi=15.93, water_area_in_sqmi=0.01, housing_units=21058, occupied_housing_units=19725, median_home_value=576200, median_household_income=65871, bounds_west=-117.23664, bounds_east=-117.107508, bounds_north=32.880848, bounds_south=32.837185)

In [5]:
#Adding new city, county, lat, lng, housing units columns  to census 
census_df["City"]=''
census_df["County"]=''
census_df['Lat']=''
census_df['Lng']=''
census_df['State']=''
for index, row in census_df.iterrows():

    # get zipcode type from df
    zipcode = int(row['Zipcode'])
    #Searching by zipcode
    search = SearchEngine()
    results = search.by_zipcode(str(zipcode))
    #adding to City columns
    try:
        census_df.loc[index, 'City'] = results.city
        census_df.loc[index, 'County'] = results.county
        census_df.loc[index, 'Lat'] = results.lat
        census_df.loc[index, 'Lng'] = results.lng
        census_df.loc[index, 'State'] = results.state
    except (KeyError, IndexError):
        print("Missing field/result for... skipping.")

In [6]:
census_df

,Zipcode,City,County,Lat,Lng,State
0,12810,Athol,Warren County,43.48,-73.88,NY
1,12811,Bakers Mills,Warren County,43.622,-74.035,NY
2,12812,Blue Mountain Lake,Hamilton County,43.9,-74.3,NY
3,12814,Bolton Landing,Warren County,43.6,-73.6,NY
4,12815,Brant Lake,Warren County,43.69,-73.71,NY
...,...,...,...,...,...,...
33115,71404,Atlanta,Winn Parish,31.8,-92.7,LA
33116,28735,Gerton,Henderson County,35.48,-82.36,NC
33117,34139,Everglades City,Collier County,25.86,-81.39,FL
33118,40007,Bethlehem,Henry County,38.45,-85.02,KY


In [35]:
groupby_data=census_df.groupby(["County", "State"]).agg(
                                {'Zipcode': lambda x : x.unique()[0]})
groupby_data.reset_index(inplace=True)  


In [36]:
groupby_data

,County,State,Zipcode
0,,OK,73047
1,Aasco Municipio,PR,00610
2,Abbeville County,SC,29620
3,Acadia Parish,LA,70516
4,Accomack County,VA,23301
...,...,...,...
3217,Yuma County,AZ,85333
3218,Yuma County,CO,80727
3219,Zapata County,TX,78067
3220,Zavala County,TX,78829


In [37]:
#groupby_data["Zipcode"]=groupby_data["Zipcode"].astype(int)


In [41]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units='metric'


In [40]:
sdzip='00610'
response=requests.get(url, params={'appid': api_key, 
                                       'zip':f"{sdzip},us", 'units':units})
weather=response.json()
print(response.url)
print(json.dumps(weather, indent=4))

http://api.openweathermap.org/data/2.5/weather?appid=1f642ecec14736cd7b6d0944b33d9cc1&zip=00610%2Cus&units=metric
{
    "coord": {
        "lon": -67.14,
        "lat": 18.28
    },
    "weather": [
        {
            "id": 802,
            "main": "Clouds",
            "description": "scattered clouds",
            "icon": "03n"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 25.48,
        "feels_like": 26.36,
        "temp_min": 24.11,
        "temp_max": 26.77,
        "pressure": 1014,
        "humidity": 87
    },
    "visibility": 10000,
    "wind": {
        "speed": 0.89,
        "deg": 70,
        "gust": 2.68
    },
    "clouds": {
        "all": 31
    },
    "dt": 1637455417,
    "sys": {
        "type": 2,
        "id": 2037648,
        "country": "US",
        "sunrise": 1637404678,
        "sunset": 1637445053
    },
    "timezone": -14400,
    "id": 0,
    "name": "Anasco",
    "cod": 200
}


In [ ]:
#converting us zipcodes to list
us_zipcodes=groupby_data["Zipcode"].to_list()
name=[]
temp_max=[]
temp_min=[]
description=[]
feels_like=[]
lon=[]
lat=[]
sunrise=[]
sunset=[]
us_zip=[]

for uszip in us_zipcodes:
    weather=requests.get(url, params={'appid': api_key, 
                                       'zip':f"{uszip},us", 'units':units}).json()
    try:
        us_zip.append(uszip)
        name.append(weather['name'])
        temp_max.append(weather['main']['temp_max'])
        temp_min.append(weather['main']['temp_min'])
        description.append(weather['weather'][0]['description'])
        feels_like.append(weather['main']['feels_like'])
        lon.append(weather['coord']['lon'])
        lat.append(weather['coord']['lat'])
        sunrise.append(weather['sys']['sunrise'])
        sunset.append(weather['sys']['sunset'])
    except:
        print(f'{us_zip} doesnot exisit')


In [52]:
weather_data=pd.DataFrame({"Zipcode":us_zip,'Name':name,'Max_temp':temp_max,
                          "Min_temp":temp_min,"Description":description,
                          "Feels_like":feels_like, 'Sunrise':sunrise,
                          "Sunset":sunset, "Lon":lon, "Lat":lat})

In [53]:
weather_data


,Zipcode,Name,Max_temp,Min_temp,Description,Feels_like,Sunrise,Sunset,Lon,Lat
0,73047,Hinton,16.27,11.07,overcast clouds,12.10,1637414050,1637450655,-98.3313,35.4675
1,00610,Anasco,27.33,24.11,scattered clouds,26.27,1637404678,1637445053,-67.1400,18.2800
2,29620,Abbeville,7.81,3.33,overcast clouds,2.23,1637410053,1637446993,-82.3785,34.1819
3,70516,Branch,18.28,14.06,clear sky,15.98,1637411983,1637449848,-92.3459,30.3446
4,23301,Accomac,7.93,2.97,overcast clouds,5.35,1637408913,1637444918,-75.6803,37.7159
5,83616,Eagle,7.47,3.20,clear sky,3.89,1637419602,1637453759,-116.3620,43.7069
6,50002,Adair,8.41,4.72,overcast clouds,4.21,1637414028,1637448907,-94.6444,41.5137
7,42728,Columbia,10.13,4.49,overcast clouds,5.00,1637411132,1637447300,-85.2656,37.1161
8,63501,Kirksville,10.65,6.99,few clouds,9.06,1637413330,1637448616,-92.5856,40.1908
9,74931,Bunch,15.59,12.11,overcast clouds,13.09,1637413220,1637449770,-94.7611,35.6826


In [55]:
final_weather=pd.merge(groupby_data, weather_data, how='inner', on='Zipcode')
final_weather

,County,State,Zipcode,Name,Max_temp,Min_temp,Description,Feels_like,Sunrise,Sunset,Lon,Lat
0,,OK,73047,Hinton,16.27,11.07,overcast clouds,12.10,1637414050,1637450655,-98.3313,35.4675
1,Aasco Municipio,PR,00610,Anasco,27.33,24.11,scattered clouds,26.27,1637404678,1637445053,-67.1400,18.2800
2,Abbeville County,SC,29620,Abbeville,7.81,3.33,overcast clouds,2.23,1637410053,1637446993,-82.3785,34.1819
3,Acadia Parish,LA,70516,Branch,18.28,14.06,clear sky,15.98,1637411983,1637449848,-92.3459,30.3446
4,Accomack County,VA,23301,Accomac,7.93,2.97,overcast clouds,5.35,1637408913,1637444918,-75.6803,37.7159
5,Ada County,ID,83616,Eagle,7.47,3.20,clear sky,3.89,1637419602,1637453759,-116.3620,43.7069
6,Adair County,IA,50002,Adair,8.41,4.72,overcast clouds,4.21,1637414028,1637448907,-94.6444,41.5137
7,Adair County,KY,42728,Columbia,10.13,4.49,overcast clouds,5.00,1637411132,1637447300,-85.2656,37.1161
8,Adair County,MO,63501,Kirksville,10.65,6.99,few clouds,9.06,1637413330,1637448616,-92.5856,40.1908
9,Adair County,OK,74931,Bunch,15.59,12.11,overcast clouds,13.09,1637413220,1637449770,-94.7611,35.6826
